In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms as T

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
sys.path

['C:\\Users\\Shyam\\Anaconda3\\envs\\unet\\python36.zip',
 'C:\\Users\\Shyam\\Anaconda3\\envs\\unet\\DLLs',
 'C:\\Users\\Shyam\\Anaconda3\\envs\\unet\\lib',
 'C:\\Users\\Shyam\\Anaconda3\\envs\\unet',
 '',
 'C:\\Users\\Shyam\\Anaconda3\\envs\\unet\\lib\\site-packages',
 'C:\\Users\\Shyam\\Anaconda3\\envs\\unet\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Shyam\\.ipython',
 'E:\\project_russel\\unet']

In [3]:
from models import UNet
from datasets import kidney
from config import config as cfg

In [4]:
train_loader = cfg.train_loader

In [5]:
tl = iter(train_loader)

In [6]:
sample = next(tl)

In [7]:
import matplotlib.pyplot as plt

label = sample['label']

In [8]:
label.size()

torch.Size([1, 400, 400])

In [10]:
import numpy as np
np.unique(label)

array([0., 1.], dtype=float32)

In [11]:
label.shape

torch.Size([1, 400, 400])